In [1]:
from helpers import *

In [2]:
main_dir = os.path.join(os.getcwd(), 'Data')
df = fetchresult(main_dir)

In [9]:
df[['INSTANCE', 'MODE']].drop_duplicates()

,INSTANCE,MODE
0,traininstance6,vpcnonbasicprojection
1,neos5,vpcnonbasicprojection
2,ran14x18-disj-8,vpcnonbasicprojection
3,30n20b8,vpcnonbasicprojection
4,seymour1,vpcnonbasicprojection
...,...,...
163,beasleyC3,vpc
164,pg,vpc
165,n5-3,vpc
166,neos-911970,vpc


In [10]:
grouped_df = df.groupby(['INSTANCE', 'MODE']).size().reset_index(name='count')

# Filter pairs that appear more than once
result = grouped_df[grouped_df['count'] > 1]
result

,INSTANCE,MODE,count


In [11]:
# Pivot the DataFrame to the desired format
df_pivot = df.pivot(index='INSTANCE', columns='MODE', values=['GAP', 'TIME','FEASIBLE'])

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]

gap_columns = [col for col in df_pivot.columns if 'GAP' in col]
print(gap_columns)
df_pivot[gap_columns] = (df_pivot[gap_columns].astype(float)*100).round(2)

['GAP_gomory', 'GAP_vpc', 'GAP_vpcnonbasicprojection', 'GAP_vpctrivialprojection']


In [6]:
df_pivot

,GAP_gomory,GAP_vpc,GAP_vpcnonbasicprojection,GAP_vpctrivialprojection,TIME_gomory,TIME_vpc,TIME_vpcnonbasicprojection,TIME_vpctrivialprojection,FEASIBLE_gomory,FEASIBLE_vpc,FEASIBLE_vpcnonbasicprojection,FEASIBLE_vpctrivialprojection
INSTANCE,,,,,,,,,,,,
30n20b8,9.06,8.62,8.62,-123.45,23.18,82.48,131.77,3700.0,False,False,True,False
beasleyC3,8.05,0.66,0.66,0.66,0.08,7.75,12.75,18.38,False,False,True,True
binkar10_1,7.32,6.15,6.15,6.15,0.1,2.97,6.8,13.24,False,False,True,True
bnatt400,0.00,0.00,0.00,-123.45,2.69,54.14,79.74,3700.0,False,False,True,False
cod105,0.89,0.44,0.44,0.44,6.24,10.38,11.98,170.09,False,False,True,True
cost266-UUE,23.54,21.18,21.18,21.18,0.59,23.88,17.86,23.33,False,False,True,True
csched007,8.68,3.39,3.39,3.39,0.22,14.35,18.21,220.4,False,False,True,True
exp-1-500-5-5,28.40,20.27,20.27,20.27,0.04,6.85,6.3,9.74,False,False,True,True
gen-ip002,2.15,0.91,0.91,0.91,0.01,2.01,3.27,3.32,False,False,True,True


In [7]:
df_pivot.to_csv("results.csv")

In [8]:
df_rounded

NameError: name 'df_rounded' is not defined